This notebook shows how to use yt to visualize the output files WarpX. For installation instructions, see here: http://yt-project.org/ .Note that WarpX currently requires the development version of yt - ask Andrew for help installing from source.

In [ ]:
import yt

We start by loading an example dataset. I generated this dataset by running the Exec/Langmuir problem with the default inputs file.

In [ ]:
ds = yt.load("Langmuir/plt00040/")

These are the default fields available for analysis:

In [ ]:
ds.field_list

Fields in yt have the form (field_type, field_name). The fields labelled "boxlib" are mesh fields, while the others are particle fields. This particular dataset has two particle species, labelled by 0 and 1. There are also the 'all' fields, which will return the data for all particle types. If you don't specify which particles you want, it will default to 'all'.

One thing you might want to do is simply grab all the particle x positions. To do so:

In [ ]:
ad = ds.all_data()
x = ad['particle_position_x']
print(x)
print(x.size)

Note that yt knows about units, so if you try to add this array to something with the wrong dimensions it will complain:

In [ ]:
ad['particle_position_x'] + ad['particle_velocity_x']

If you only wanted a certain species, you could do:

In [ ]:
m = ad['particle0', 'particle_mass']
print(m)
print(m.size)

or

In [ ]:
m = ad['particle1', 'particle_mass']
print(m)
print(m.size)

If your dataset is large, you probably don't want to read all the particles in at once. You can also grid-by-grid. To see all the grids in your dataset, do:

In [ ]:
ds.index.grids

To look at the particles only in the one grid, we do:

In [ ]:
g = ds.index.grids[0]
x = g['particle_velocity_x']
print(x)
print(x.size)

You can also define arbitrary geometric regions for selecting data. Below, we get the weights of particles within a sphere of given position and radius

In [ ]:
center = ds.domain_center
radius = ds.domain_width[0] / 4.0
sp = ds.sphere(center, radius)
w = sp['particle_weight']
print(w)
print(w.size)

Grids that do not intersect with the geometric object are never loaded into memory. There are a bunch of other geometric constructs you can use - rectangular regions, disks, slices, etc...

You can also define derived fields that build off of the basic ones. Below, we show how to create a derived field called "particle_gamma" for every particle type in the simulation. Commonly used derived fields can be added to the WarpX frontend, so you don't have to do this over and over in your scripts.

In [ ]:
from yt.utilities.physical_constants import c
import numpy as np

def add_particle_gamma(ds, ptype):
    def _gamma(field, data):
        vx = data[ptype, 'particle_velocity_x']
        vy = data[ptype, 'particle_velocity_y']
        vz = data[ptype, 'particle_velocity_z']
        v2 = vx**2 + vy**2 + vz**2
        c2 = c**2
        return 1.0/np.sqrt(1.0 - v2/c2)
    ds.add_field((ptype, 'particle_gamma'), _gamma, sampling_type='particle')
    
for ptype in ds.particle_types:
    add_particle_gamma(ds, ptype)
    
print(ad['all', 'particle_gamma'])

yt has built-in plotting functions for quickly making plots of your dataset. These plots are AMR-aware and have lots of callback functions for customizing them to your liking. 

Below, we show how to take a slice perpindicular to the y direction through the simulation domain, showing the current in the x-direction. We have also over-plotted the particle positions in a one-cell thick region near the slice. This example uses a different dataset, generated my running the uniform_plasma example with the default inputs file, except that plotfiles were turned on.

In [ ]:
direction = 1
field = 'jx'

ds = yt.load("../Exec/uniform_plasma/plt00000/")
dx = ds.domain_width[direction] / ds.domain_dimensions[direction]
sl = yt.SlicePlot(ds, direction, field)
sl.annotate_particles(width=dx, col='k', marker='.', p_size=1.0)

Here is a slighly different example that uses the plasma_acceleration test problem. We only overplot the particle positions for species 1.

In [ ]:
direction = 1
field = 'jx'

ds = yt.load("../Exec/plasma_acceleration/plt00040/")
dx = ds.domain_width[direction] / ds.domain_dimensions[direction]
sl = yt.SlicePlot(ds, direction, field)
sl.annotate_particles(width=dx, ptype='particle1', col='k', marker='.', p_size=10.0)

Of course, once the particles are read in, you can also just use matplotlib to make your plots. Here is an example that makes an interactive 3D plot showing the particle positions of species 0 from the above simulation.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
%matplotlib

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ds = yt.load("../Exec/plasma_acceleration/plt00040/")
ad = ds.all_data()
ptype = 'particle0'
xs = ad[(ptype, 'particle_position_x')]
ys = ad[(ptype, 'particle_position_y')]
zs = ad[(ptype, 'particle_position_z')]

ax.scatter(xs, ys, zs, marker='.')

There is a bunch more you can do with yt, please see the documentation here: http://yt-project.org/doc/index.html and the website here: http://yt-project.org/ for more information. 